In [13]:
import pandas as pd

# Load the datasets (replace with your actual file paths)
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

# Merge transactions with product information to get 'Price' and 'Category' for each transaction
transactions_df = transactions_df.merge(products_df[['ProductID', 'Price', 'Category']], on='ProductID', how='left')

# Rename columns to avoid confusion between 'Price' columns
transactions_df.rename(columns={'Price_y': 'ProductPrice'}, inplace=True)  # Rename 'Price_y' to 'ProductPrice'
transactions_df.drop(columns=['Price_x'], inplace=True)  # Drop 'Price_x' which is redundant

# Fill missing values in 'Category' with 'Unknown'
transactions_df['Category'] = transactions_df['Category'].fillna('Unknown')

# Aggregating customer transaction data
customer_transactions = transactions_df.groupby('CustomerID').agg(
    total_transactions=('TransactionID', 'count'),       # Count of transactions
    total_spent=('TotalValue', 'sum'),                   # Total amount spent
    average_price=('ProductPrice', 'mean'),              # Average price of items bought
    most_frequent_category=('Category', lambda x: x.mode()[0] if not x.mode().empty else 'Unknown')  # Handle empty mode
).reset_index()

# Merge the aggregated transaction data with customer profile information
customer_profile = customers_df[['CustomerID', 'Region']].merge(customer_transactions, on='CustomerID', how='left')

# One-hot encode categorical features (Region, most_frequent_category)
customer_profile = pd.get_dummies(customer_profile, columns=['Region', 'most_frequent_category'])

# Display customer profile with aggregated transaction features
print(customer_profile.head())


  CustomerID  total_transactions  total_spent  average_price  Region_Asia  \
0      C0001                 5.0      3354.52     278.334000        False   
1      C0002                 4.0      1862.74     208.920000         True   
2      C0003                 4.0      2725.38     195.707500        False   
3      C0004                 8.0      5354.88     240.636250        False   
4      C0005                 3.0      2034.24     291.603333         True   

   Region_Europe  Region_North America  Region_South America  \
0          False                 False                  True   
1          False                 False                 False   
2          False                 False                  True   
3          False                 False                  True   
4          False                 False                 False   

   most_frequent_category_Books  most_frequent_category_Clothing  \
0                         False                            False   
1               

In [17]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# Ensure no missing values in the customer profile features used for scaling
# Check for missing values
print(customer_profile.isnull().sum())

# Option 1: Fill NaN values with 0 (or another strategy like the column mean)
customer_features = customer_profile.drop(columns='CustomerID')
customer_features.fillna(0, inplace=True)  # Filling NaN with 0

# Option 2: Alternatively, drop rows with missing values
# customer_features.dropna(inplace=True)

# Standardize the features (important for cosine similarity)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features)

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(scaled_features)

# Create a DataFrame to hold the similarity scores
cosine_sim_df = pd.DataFrame(cosine_sim, index=customer_profile['CustomerID'], columns=customer_profile['CustomerID'])

# Display the similarity matrix (only the first 10 customers for readability)
print(cosine_sim_df.iloc[:10, :10])


CustomerID                            0
total_transactions                    1
total_spent                           1
average_price                         1
Region_Asia                           0
Region_Europe                         0
Region_North America                  0
Region_South America                  0
most_frequent_category_Books          0
most_frequent_category_Clothing       0
most_frequent_category_Electronics    0
most_frequent_category_Home Decor     0
dtype: int64
CustomerID     C0001     C0002     C0003     C0004     C0005     C0006  \
CustomerID                                                               
C0001       1.000000 -0.289569  0.232114  0.208178  0.353981  0.257408   
C0002      -0.289569  1.000000 -0.112930 -0.399363  0.385806 -0.420811   
C0003       0.232114 -0.112930  1.000000  0.138598 -0.216224  0.124638   
C0004       0.208178 -0.399363  0.138598  1.000000 -0.503516  0.686392   
C0005       0.353981  0.385806 -0.216224 -0.503516  1.000000 -0

In [19]:
# Function to get top N lookalikes for a given customer
def get_top_lookalikes(customer_id, top_n=3):
    similarity_scores = cosine_sim_df[customer_id].sort_values(ascending=False)[1:]  # Exclude self-similarity
    top_lookalikes = similarity_scores.head(top_n)
    return top_lookalikes.index.tolist(), top_lookalikes.values.tolist()

# Get top 3 lookalikes for each of the first 20 customers
lookalikes = {}
for customer_id in customer_profile['CustomerID'].iloc[:20]:
    lookalikes[customer_id] = get_top_lookalikes(customer_id)

# Create the Lookalike.csv file
lookalike_list = []
for cust_id, (similar_cust_ids, scores) in lookalikes.items():
    for sim_cust_id, score in zip(similar_cust_ids, scores):
        lookalike_list.append({'cust_id': cust_id, 'lookalike_cust_id': sim_cust_id, 'similarity_score': score})

# Convert to DataFrame and save to CSV
lookalike_df = pd.DataFrame(lookalike_list)
lookalike_df.to_csv('Lookalike.csv', index=False)

# Display the first few rows of the Lookalike.csv
print(lookalike_df.head())


  cust_id lookalike_cust_id  similarity_score
0   C0001             C0181          0.985151
1   C0001             C0192          0.958554
2   C0001             C0190          0.957915
3   C0002             C0088          0.985121
4   C0002             C0106          0.966302


In [23]:
import os
print(os.listdir())  # Lists all files in the current working directory


['.anaconda', '.apexlink_cache', '.cache', '.conda', '.condarc', '.continuum', '.eclipse', '.idlerc', '.ipynb_checkpoints', '.ipython', '.jupyter', '.keras', '.m2', '.matplotlib', '.p2', '.packettracer', '.redhat', '.sf', '.sfdx', '.virtual_documents', '.vscode', 'anaconda3', 'AppData', 'Application Data', 'Cisco Packet Tracer 8.2.2', 'Contacts', 'Cookies', 'Customers.csv', 'data.csv', 'Documents', 'Downloads', 'eclipse', 'eclipse-workspace', 'Favorites', 'Links', 'Local Settings', 'Lookalike.csv', 'Music', 'My Documents', 'NetHood', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{a2332f17-cdbf-11ec-8680-002248483d79}.TxR.0.regtrans-ms', 'NTUSER.DAT{a2332f17-cdbf-11ec-8680-002248483d79}.TxR.1.regtrans-ms', 'NTUSER.DAT{a2332f17-cdbf-11ec-8680-002248483d79}.TxR.2.regtrans-ms', 'NTUSER.DAT{a2332f17-cdbf-11ec-8680-002248483d79}.TxR.blf', 'NTUSER.DAT{a2332f18-cdbf-11ec-8680-002248483d79}.TM.blf', 'NTUSER.DAT{a2332f18-cdbf-11ec-8680-002248483d79}.TMContainer000000000000000000

In [27]:
# Read the saved CSV file to confirm its content
lookalike_df = pd.read_csv('Lookalike.csv')
print(lookalike_df.head())  # Display the first few rows of the Lookalike recommendations


  cust_id lookalike_cust_id  similarity_score
0   C0001             C0181          0.985151
1   C0001             C0192          0.958554
2   C0001             C0190          0.957915
3   C0002             C0088          0.985121
4   C0002             C0106          0.966302


In [29]:
import shutil

# Move the file to the current directory if not already
shutil.move('Lookalike.csv', 'Lookalike.csv')


'Lookalike.csv'